In [333]:
import requests
import numpy as np
from bs4 import BeautifulSoup
import re
import pandas as pd 

Idea here is a function that takes a URL and returns a dictionary from country to number of cases, deaths, and recoveries. This is pretty terrible but it's a start.

In [468]:
def cleanCountry(string):
    newString = re.sub(r'\[.*\]', '', string)
    newString = re.sub(r'\(.*\)', '', newString)
    if len(newString)>1:
        if newString[0]==" ":
            newString=newString[1:]
        if newString[-1]==" ":
            newString=newString[:-1]
    return newString
    

In [729]:
def cleanNumber(string):
    return int(cleanCountry(string).replace(",","").replace(".","").replace("–","0").replace("?","0").replace("-","0").replace("~","0"))

In [598]:
def getDictionaryFromWikiTable(url):
    website_url = requests.get(url).text
    soup = BeautifulSoup(website_url,"html.parser")
    My_table = soup.find("table",{"class":"wikitable plainrowheaders sortable"})
    if type(My_table) != bs4.element.Tag:
        My_table = soup.find("table",{"class":"wikitable plainrowheaders sortable mw-collapsible"})
    #return My_table
    
    # Find countries or things like them
    links=My_table.findAll("tr")
    countries=[]
    start=0
    for link in links:
        #if len(link.findAll('th'))==2:
        if start==0:
            if "China" in cleanCountry(link.findAll('th')[-1].text[:-1]):
                start=1
        if start==1:
            if len(link.findAll('td')) < 3:
                break
            country = cleanCountry(link.findAll('th')[-1].text[:-1])
            nums = [cleanNumber(link.findAll('td')[i].text[:-1]) for i in range(3)]
            countries.append([country]+nums)
    
    #Generate dictionary
    myDict = {}
    for i in range(len(countries)):
        myDict[countries[i][0]]=countries[i][1:]
    return myDict

Let's find the URLs of all the pages that we want to scrape.

In [573]:
#history = "https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&offset=&limit="+str(10000)+"&action=history"
history="https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&offset=20200301201057&limit=5000&action=history"
website_url = requests.get(history).text
soupTemp = BeautifulSoup(website_url,"html.parser")
    

In [607]:
def getUrls(dates,limit):
    # Ensure limit is really big
    history = "https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&offset=&limit="+str(limit)+"&action=history"
    website_url = requests.get(history).text
    soup = BeautifulSoup(website_url,"html.parser")
    My_table = soup.find("form",{"id":"mw-history-compare"})
    #My_table = soupTemp.find("form",{"id":"mw-history-compare"})
    datesloc=My_table.findAll("a")
    #return datesloc
    urls=[]
    for datetext in dates:
        versions=[]
        for date in datesloc:
            if datetext in date.text:
                versions.append(date["href"])
        if len(versions)>0:
            urls.append("https://en.wikipedia.org"+versions[-1])
        else:
            print("date not found")
    return urls

In [720]:
getUrls(["20 March 2020"],500)

['https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=946409034']

In [347]:
datestext=[str(i)+" March 2020" for i in range(1,19)]

In [524]:
datestext=[str(i)+" February 2020" for i in range(1,30)]

In [527]:
datestext

['1 February 2020',
 '2 February 2020',
 '3 February 2020',
 '4 February 2020',
 '5 February 2020',
 '6 February 2020',
 '7 February 2020',
 '8 February 2020',
 '9 February 2020',
 '10 February 2020',
 '11 February 2020',
 '12 February 2020',
 '13 February 2020',
 '14 February 2020',
 '15 February 2020',
 '16 February 2020',
 '17 February 2020',
 '18 February 2020',
 '19 February 2020',
 '20 February 2020',
 '21 February 2020',
 '22 February 2020',
 '23 February 2020',
 '24 February 2020',
 '25 February 2020',
 '26 February 2020',
 '27 February 2020',
 '28 February 2020',
 '29 February 2020']

In [577]:
Urlstext=getUrls(datestext,100)

In [605]:
Urlstext

['https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=938566085',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=938722769',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=938886445',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=939041138',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=939201335',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=939362116',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=939524881',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=939679762',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandem

Now scrape those URLs

In [ ]:
FebUrlstext=['https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=938566085',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=938722769',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=938886445',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=939041138',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=939201335',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=939362116',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=939524881',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=939679762',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=939833918',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=940002901',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=940174916',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=940342980',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=940512502',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=940683119',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=940852174',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=941010764',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=941175150',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=941358392',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=941506785',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&direction=next&oldid=941684122',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=941848218',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=942002862',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=942161086',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=942326494',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=942496885',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=942649494',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=942813905',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=942965838',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=943124253']

In [721]:
MarUrlstext=['https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=943311597',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=943454674',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=943633545',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=943799750',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=943984373',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=944142051',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=944306871',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=944460601',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=944630177',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=944799518',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=944965160',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=945127664',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=945288396',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=945439727',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=945597450',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=945762744',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=945925005',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=946086474',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=946244618',
 'https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=946409034']

In [603]:
allDicts = []
for urls in Urlstext:
    try:
        allDicts.append(getDictionaryFromWikiTable(urls))
    except:
        print(urls)

https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=938566085
https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=938722769
https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=938886445
https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=939041138
https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=939201335
https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=939362116
https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=939524881
https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=939679762
https://en.wikipedia.org/w/index.php?title=Template:2019%E2%80%9320_coronavirus_pandemic_data&oldid=939833918
https://en

In [470]:
allKeys=np.sort(list(set.union(*[set(ad.keys()) for ad in allDicts])))

In [474]:
len(allKeys)

158

In [475]:
[len(ad) for ad in allDicts]

[65, 68, 77, 81, 85, 87, 96, 102, 103, 108, 112, 116, 120, 134, 137, 142, 147]

In [476]:
allKeys

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Bangladesh', 'Belarus', 'Belgium', 'Benin', 'Bhutan', 'Bolivia',
       'Bosnia and Herzegovina', 'Brazil', 'Brunei', 'Bulgaria',
       'Burkina Faso', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chile', 'China', 'Colombia',
       'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czech Republic',
       'DR Congo', 'Denmark', 'Diamond Princess', 'Dominican Republic',
       'Ecuador', 'Egypt', 'Equatorial Guinea', 'Estonia', 'Eswatini',
       'Ethiopia', 'Finland', 'France', 'Gabon', 'Georgia', 'Germany',
       'Ghana', 'Gibraltar', 'Grand Princess', 'Greece', 'Guatemala',
       'Guernsey', 'Guinea', 'Guyana', 'Honduras', 'Hong Kong', 'Hungary',
       'Iceland', 'India', 'Indonesia', 'International\xa0conveyance',
       'Iran', 'Iraq', 'Ireland', 'Israel', 'Italy', 'Ivo

In [622]:
allKeys=['Afghanistan', 'Albania', 'Algeria', 'Andorra',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Bangladesh', 'Belarus', 'Belgium', 'Benin', 'Bhutan', 'Bolivia',
       'Bosnia and Herzegovina', 'Brazil', 'Brunei', 'Bulgaria',
       'Burkina Faso', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chile', 'China', 'Colombia',
       'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czech Republic',
       'DR Congo', 'Denmark', 'Diamond Princess', 'Dominican Republic',
       'Ecuador', 'Egypt', 'Equatorial Guinea', 'Estonia', 'Eswatini',
       'Ethiopia', 'Finland', 'France', 'Gabon', 'Georgia', 'Germany',
       'Ghana', 'Gibraltar', 'Grand Princess', 'Greece', 'Guatemala',
       'Guernsey', 'Guinea', 'Guyana', 'Honduras', 'Hong Kong', 'Hungary',
       'Iceland', 'India', 'Indonesia',
       'Iran', 'Iraq', 'Ireland', 'Israel', 'Italy', 'Ivory Coast',
       'Jamaica', 'Japan', 'Jersey', 'Jordan', 'Kazakhstan', 'Kenya',
       'Kosovo', 'Kuwait', 'Latvia', 'Lebanon', 'Liberia',
       'Liechtenstein', 'Lithuania', 'Luxembourg',
       'Macau', 'Malaysia', 'Maldives', 'Malta',
       'Mauritania', 'Mexico', 'Moldova', 'Monaco', 'Mongolia', 'Morocco',
       'Namibia', 'Nepal', 'Netherlands', 'New Zealand', 'Nigeria',
       'North Korea', 'North Macedonia', 'Northern Cyprus', 'Norway',
       'Oman', 'Pakistan', 'Palestine', 'Panama', 'Paraguay', 'Peru',
       'Philippines', 'Poland', 'Portugal', 'Qatar',
       'Republic of the Congo', 'Romania', 'Russia', 'Rwanda',
       'Saint Lucia', 'San Marino', 'Saudi Arabia', 'Senegal', 'Serbia',
       'Seychelles', 'Singapore', 'Slovakia', 'Slovenia', 'Somalia',
       'South Africa', 'South Korea', 'Spain', 'Sri Lanka', 'St. Vincent and the Grenadines',
       'Sudan', 'Suriname', 'Sweden', 'Switzerland', 'Taiwan', 'Tanzania',
       'Thailand', 'Togo', 'Trinidad and Tobago', 'Tunisia', 'Turkey',
       'Ukraine', 'United Arab Emirates', 'United Kingdom',
       'United States', 'Uruguay', 'Uzbekistan', 'Vatican City',
       'Venezuela', 'Vietnam']

In [580]:
allDicts[0]['United States']

[13, 0, 3]

In [522]:
def getData(dictIndex,key):
    try:
        output = allDicts[dictIndex][key]
    except:
        output = ["","",""]
    return output

In [615]:
def getData2(dic,key):
    try:
        output = dic[key]
    except:
        output = ["","",""]
    return output

In [604]:
data=np.array([np.concatenate([getData(i,key) for key in allKeys]) for i in range(len(allDicts))])

header1=np.concatenate([[key,"",""] for key in allKeys])

header2=np.concatenate([["cases","dead","recovered"] for key in allKeys])

allData=np.concatenate(((header1,header2),data),axis=0)

np.savetxt("/home/handmer/Documents/Projects/Covid/COVID19numbersFeb.csv",allData,delimiter=",", fmt='%s', comments='',)

For updating the daily numbers

In [644]:
todayKeys=['Afghanistan', 'Albania', 'Algeria', 'Andorra',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Benin', 'Bhutan',
       'Bolivia', 'Bosnia and Herzegovina', 'Brazil', 'Brunei',
       'Bulgaria', 'Burkina Faso', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chile', 'China', 'Colombia',
       'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czech Republic',
       'DR Congo', 'Denmark', 'Diamond Princess', 'Djibouti',
       'Dominican Republic', 'Ecuador', 'Egypt', 'Equatorial Guinea',
       'Estonia', 'Eswatini', 'Ethiopia', 'Finland', 'France', 'Gabon',
       'Gambia', 'Georgia', 'Germany', 'Ghana', 'Gibraltar',
       'Grand Princess', 'Greece', 'Guatemala', 'Guernsey', 'Guinea',
       'Guyana', 'Honduras', 'Hong Kong', 'Hungary', 'Iceland', 'India',
       'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Israel', 'Italy',
       'Ivory Coast', 'Jamaica', 'Japan', 'Jersey', 'Jordan',
       'Kazakhstan', 'Kenya', 'Kosovo', 'Kuwait', 'Kyrgyzstan', 'Latvia',
       'Lebanon', 'Liberia', 'Liechtenstein', 'Lithuania', 'Luxembourg',
       'Macau', 'Malaysia', 'Maldives', 'Malta', 'Mauritania',
       'Mauritius', 'Mexico', 'Moldova', 'Monaco', 'Mongolia',
       'Montenegro', 'Morocco', 'Namibia', 'Nepal', 'Netherlands',
       'New Zealand', 'Nigeria', 'North Korea', 'North Macedonia',
       'Northern Cyprus', 'Norway', 'Oman', 'Pakistan', 'Palestine',
       'Panama', 'Paraguay', 'Peru', 'Philippines', 'Poland', 'Portugal',
       'Qatar', 'Republic of the Congo', 'Romania', 'Russia', 'Rwanda',
       'Saint Lucia', 'San Marino', 'Saudi Arabia', 'Senegal', 'Serbia',
       'Seychelles', 'Singapore', 'Slovakia', 'Slovenia', 'Somalia',
       'South Africa', 'South Korea', 'Spain', 'Sri Lanka',
       'St. Vincent and the Grenadines', 'Sudan', 'Suriname', 'Sweden',
       'Switzerland', 'Taiwan', 'Tanzania', 'Thailand', 'Togo',
       'Trinidad and Tobago', 'Tunisia', 'Turkey', 'Ukraine',
       'United Arab Emirates', 'United Kingdom', 'United States',
       'Uruguay', 'Uzbekistan', 'Vatican City', 'Venezuela', 'Vietnam',
       'Zambia']

In [724]:
today = getDictionaryFromWikiTable(MarUrlstext[-1])

In [725]:
set(today.keys())-set(todayKeys)

{'Chad', 'El Salvador', 'Fiji', 'Nicaragua', 'Niger'}

In [726]:
set(todayKeys)-set(today.keys())

{'Aruba', 'Gibraltar', 'Grand Princess', 'Guernsey', 'Jersey', 'North Korea'}

In [727]:
todayKeys=np.sort(list(set(today.keys())|set(todayKeys)))

In [728]:
data=np.concatenate([getData2(today,key) for key in todayKeys])

header1=np.concatenate([[key,"",""] for key in todayKeys])

header2=np.concatenate([["cases","dead","recovered"] for key in todayKeys])

allData=np.array([header1,header2,data])

np.savetxt("/home/handmer/Documents/Projects/Covid/COVID19numbersToday.csv",allData,delimiter=",", fmt='%s', comments='',)

Let's crunch the US data from covidtracking.com

In [730]:
import csv

USData=[]

with open("/home/handmer/Documents/Projects/Covid/COVIDUSA.csv") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        USData.append(row)
        #if line_count == 0:
        #    print(f'Column names are {", ".join(row)}')
        #    line_count += 1
        #else:
        #    print(row)
        #    line_count += 1
    #print(f'Processed {line_count} lines.')

In [731]:
dates=np.sort(list(set([x[0] for x in USData[1:]])))

In [732]:
states=np.sort(list(set([x[1] for x in USData[1:]])))

In [733]:
dates

array(['20200304', '20200305', '20200306', '20200307', '20200308',
       '20200309', '20200310', '20200311', '20200312', '20200313',
       '20200314', '20200315', '20200316', '20200317', '20200318',
       '20200319'], dtype='<U8')

In [735]:
def generateDictFromDate(date):
    myDict={}
    for x in USData:
        if x[0]==date:
            out=[]
            for y in x[2:]:
                try:
                    out.append(int(y))
                except:
                    out.append(0)
            myDict[x[1]]=out
    return myDict

In [736]:
USDicts=[generateDictFromDate(d) for d in dates]

In [737]:
USDicts[0]['AZ']

[2, 27, 5, 0, 34]

In [738]:
def getData3(dic,key):
    try:
        output = dic[key]
    except:
        output = ["","","","",""]
    return output

In [739]:
USData[0][2:]

['Positive', 'Negative', 'Pending', 'Death', 'Total']

In [740]:
data2=np.array([np.concatenate([getData3(USD,key) for key in states]) for USD in USDicts])
header1=np.concatenate([[key,"","","",""] for key in states])
header2=np.concatenate([USData[0][2:] for key in states])
allData2=np.concatenate(((header1,header2),data2),axis=0)
np.savetxt("/home/handmer/Documents/Projects/Covid/COVID19USData.csv",allData2,delimiter=",", fmt='%s', comments='',)